In [1]:
import os 
import sys
import pandas as pd
import time
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json
import datetime
import sqlite3
import logging
%load_ext autoreload
%autoreload 2

In [2]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time().strftime('%H:%M:%S'))
print(current_dt)

2020-01-23 00:14:39.111806


In [3]:
link_homepage = 'https://meh.com'
page = requests.get(link_homepage)
print('Status Code', page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')

Status Code 200


## Save Raw Response

In [26]:
# os.chdir('..')
db_location = 'data/meh_scraper_qa.db'

In [72]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [73]:
stats = soup.find(class_='community-stats')

In [74]:
stats1 = str(stats)
stats2 = stats1.replace('\'','\'\'')

In [75]:
cursor.execute("""INSERT INTO raw_site_community_stats (date, time, raw_site) 
        VALUES ('{0}', '{1}', '{2}')""".format(current_date, current_time, stats2))

In [76]:
con.commit()

In [77]:
con.close()

## Parse Wanted Fields

In [10]:
parsed_nums = {}
parsed_nums['date'] = current_date
parsed_nums['time'] = current_time

### Page Views

In [11]:
page_views = stats.find('div', {'id':'pageviews'})

In [12]:
page_view_secondary = page_views.find(class_='secondary').get_text()
secondary_parts = page_view_secondary.rstrip().lstrip().split(', ')
for visit_method, method_num in zip(secondary_parts, range(1,3)):
    visit_parts = visit_method.split('% ')
    parsed_nums['Page Views ' + str(method_num)+' Name'] = visit_parts[1]
    parsed_nums['Page Views ' + str(method_num)+' Percent'] = int(visit_parts[0])

In [13]:
primary_class = page_views.find_all(class_='primary')
primary_numbs = [x.find('strong').get_text() for x in primary_class]
parsed_nums['# Visited'] = int(primary_numbs[0])
parsed_nums['Clicked Meh'] = int(primary_numbs[1])


In [14]:
parsed_nums

{'date': '2020-01-23',
 'time': '00:14:39',
 'Page Views 1 Name': 'on a phone',
 'Page Views 1 Percent': 62,
 'Page Views 2 Name': 'on a tablet.',
 'Page Views 2 Percent': 5,
 '# Visited': 2715,
 'Clicked Meh': 712}

### Referrals

In [15]:
referrals = stats.find('div', {'id':'referrals'})

In [16]:
parsed_nums['Typed Meh Percent'] = int(referrals.find(class_='primary').find('strong').get_text()[:-1])

In [17]:
parsed_nums['Referrals Percent'] = int(referrals.find(class_='secondary').find('strong').get_text()[:-1])

In [18]:
referrers = referrals.find_all(class_='referrer')
for ref, ref_num in zip(referrers, range(1, len(referrers)+1)):
    parsed_nums['Referrer '+str(ref_num) + ' Percent'] = float(ref['data-percentage'])
    parsed_nums['Referrer '+str(ref_num) + ' Name'] = ref.get_text().lstrip().rstrip().split(' \n')[0]
    

### Order Counts

In [19]:
order_stats = stats.find('div', {'id': 'order-stats'})

In [20]:
order_nums = [re.findall('\d+', x.get_text())[0] for x in order_stats.find_all(class_='primary')]

In [21]:
parsed_nums['Items Sold'] = order_nums[0]
parsed_nums['Dollars Sold'] = order_nums[1]

## Export

In [79]:
export_series = pd.DataFrame(parsed_nums, index=[1])

In [80]:
export_series

,date,time,Page Views 1 Name,Page Views 1 Percent,Page Views 2 Name,Page Views 2 Percent,# Visited,Clicked Meh,Typed Meh Percent,Referrals Percent,Referrer 1 Percent,Referrer 1 Name,Items Sold,Dollars Sold
1,2020-01-23,00:14:39,on a phone,62,on a tablet.,5,2715,712,95,5,1.621622,twitter.com,2,142


In [83]:
con = sqlite3.connect(db_location)
cursor = con.cursor()
export_series.to_sql('selling_details', con, if_exists='append', index=False)

In [84]:
con.commit()
con.close()